In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import re
import string
from copy import deepcopy

In [2]:
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

In [3]:
from gensim import corpora, models, similarities, matutils

In [4]:
import nltk
from nltk import word_tokenize 
from nltk.stem import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/charlieyaris/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
class LemmaStemTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        self.ss = SnowballStemmer('english')
    
    def __call__(self, text):
        words = re.sub(r"[^A-Za-z0-9\-]", " ", text).lower().split()
        lemma_word = [self.wnl.lemmatize(t, pos = 'v') for t in word_tokenize(str(words))]
        words = [self.ss.stem(word) for word in lemma_word]
        return words

In [7]:
def replace(string, substitutions):
    for i, substitution in enumerate(substitutions):
        string = deepcopy(([s.replace(list(substitutions.keys())[i], list(substitutions.values())[i]) for s in string]))
    return string

In [8]:
pd.set_option('display.max_columns', 100000)

In [9]:
df = pd.read_pickle('prophecy_df.pkl')

In [10]:
df['century'].value_counts(normalize = True)

10.0    0.106157
9.0     0.106157
8.0     0.106157
6.0     0.106157
5.0     0.106157
4.0     0.106157
3.0     0.106157
2.0     0.106157
1.0     0.106157
7.0     0.044586
Name: century, dtype: float64

In [11]:
substitutions = {'african': 'africa', 'barbaric': 'barbarian', 'blood': 'bleed', 'bloody': 'bleed',
                 'bled': 'bleed', 'burnt': 'burn', 'captive': 'capture', 'captive': 'captured',
                 'children': 'child', 'complaints': 'complain', 'dead': 'die', 'death': 'die',
                 'deceive': 'deceit', 'depth': 'deep', 'defense': 'defend', 'destruction': 'destroy',
                 'discovered': 'discover', 'discoveries': 'discover', 'discovery': 'discover',
                 'doubly': 'double', 'drawn': 'draw', 'dreamer': 'dream', 'driven': 'drive',
                 'drunken': 'drunk', 'eastern': 'east', 'eaten': 'eat', 'enclosed': 'enclose',
                 'enclosure': 'enclose', 'entry': 'enter', 'envied': 'envy', 'envious': 'envy',
                 'executioners': 'execute', 'extension': 'extend', 'falsify': 'false', 'failure': 'fail',
                 'flight': 'flies', 'florense': 'florence', 'french': 'france', 'fraudulent': 'fraud',
                 'friendship': 'friend', 'frightful': 'frighten', 'frost': 'freeze', 'frozen': 'freeze',
                 'furious': 'fury', 'gascony': 'gascon', 'genevans': 'geneva', 'gnashing': 'gnaw',
                 'governor': 'govern', 'greedy': 'greed', 'germany': 'german', 'golden': 'gold',
                 'greatest': 'great', 'greater': 'great', 'halfway': 'half', 'healthy': 'health',
                 'hairy': 'hair', 'hardships': 'hard', 'hatred': 'hate', 'heard': 'hear', 'higher': 'high',
                 'highest': 'high', 'household': 'house', 'hungarians': 'hungary', 'impetuosity': 'impetuous',
                 'injured': 'injure', 'italian': 'italy', 'judges': 'judge', 'judgement': 'judge',
                 'kingdom': 'king', 'larger': 'large', 'later': 'late', 'leadership': 'lead', 'leader': 'lead',
                 'leaderless': 'lead', 'leaguers': 'league', 'longer': 'long', 'loss': 'lose',
                 'lost': 'lose', 'lower': 'low', 'lowest': 'low', 'lover': 'love', 'lusitanian': 'lusitania',
                 'macedonian': 'macedonia', 'mankind': 'man', 'marriage': 'marry', 'maritime': 'marine',
                 'mightily': 'mighty', 'newly': 'new', 'older': 'old', 'oldest': 'old', 'oppose': 'opposite',
                 'pleasing': 'pleasure', 'philosophers': 'philosophy', 'provincial': 'province',
                 'pursuers': 'pursue', 'pursuit': 'pursue', 'robbery': 'rob', 'robber': 'rob',
                 'redden': 'red', 'redbeard': 'red', 'remainder': 'remain', 'revolutions': 'revolt',
                 'robbed': 'rob', 'shadow': 'shade', 'seventh': 'seven', 'shorter': 'short',
                 'smaller': 'small', 'soldiery': 'soldiers', 'sovereignty': 'sovereign', 'spacious': 'space',
                 'spanish': 'spain', 'spaniards': 'spain', 'strongest': 'strong', 'stronghold': 'strong',
                 'survivor': 'survive', 'swampy': 'swamps', 'sprung': 'spring', 'taken': 'take',
                 'taker': 'take', 'terrified': 'terror', 'terrible': 'terror', 'thirsty': 'thirst',
                 'thought': 'think', 'threatens': 'threat', 'toulousans': 'toulouse', 'thunderbolt': 'thunder',
                 'treacherous': 'treachery', 'trickery': 'trick', 'tuscany': 'tuscan', 'twentieth': 'twenty',
                 'tyrant': 'tyranny', 'victorious': 'victor', 'victory': 'victor', 'violent': 'violence',
                 'younger': 'young', 'warlike': 'war', 'weak': 'weaken', 'weaker': 'weaken',
                 'wooden': 'wood', 'worldwide': 'world', 'worthy': 'worth', 'youth': 'young'}

df['prophecy'] = deepcopy(replace(df['prophecy'], substitutions))

In [12]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

df['prophecy'] = df['prophecy'].map(alphanumeric).map(punc_lower)

In [13]:
df.head()

,century,prophecy,prophecy_num
0,1.0,sitting alone at night in secret study it is...,1.0
1,1.0,the wand in the hand is placed in the middle o...,2.0
2,1.0,when the litters are overturned by the whirlwi...,3.0
3,1.0,in the world there will be made a king who wi...,4.0
4,1.0,they will be drive away for a long draw out fi...,5.0


In [14]:
lemma = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
tokenized_list = []
for i in range(0, len(df['prophecy']) - 1):
    for original_word in df['prophecy'][i].split():
        word = re.sub(r"[^A-Za-z0-9\-]", " ", original_word).lower().split()
        lemma_word = [lemma.lemmatize(t, pos = 'v') for t in word_tokenize(word[0])][0]
        stem_word = [stemmer.stem(str(lemma_word))][0]
        tokenized_list.append(stem_word + ", " + original_word)
        
tokenized_df = pd.DataFrame(tokenized_list)

In [17]:
tokenized_df.head()

,0
0,"sit, sitting"
1,"alon, alone"
2,"at, at"
3,"night, night"
4,"in, in"


In [18]:
tokenized_df[1] = [item[1] for item in tokenized_df[0].str.split(', ')]
tokenized_df[0] = [item[0] for item in tokenized_df[0].str.split(', ')]
tokenized_df.rename({0: 'stem', 1: 'original_word'}, axis = 1, inplace = True)
tokenized_df['original_word_copy'] = deepcopy(tokenized_df['original_word'])

In [19]:
aggregations = {
    'original_word_copy': 'count'
    }

tokenized_df_group = tokenized_df.groupby(['stem', 'original_word']).agg(aggregations).reset_index()

In [20]:
duplicate_list = ['stem']

tokenized_df_group.drop_duplicates(subset = duplicate_list, keep = 'first', inplace = True)

In [21]:
tokenized_df_group = tokenized_df_group.reset_index()
tokenized_df_group.drop('index', axis = 1, inplace = True)

In [22]:
stop_word_list = ['able', 'according', 'aix', 'la', 'dy', 'having', 'l', 'le',
                  'li', 'o', 's', "’", 'very', 'like', 'sens', 'did', "'",
                  "'a", "'i", "'k", "'l", "'o", "'s", ',', '[', ']']
stop_words = text.ENGLISH_STOP_WORDS.union(stop_word_list)
tf_cv1 = TfidfVectorizer(tokenizer = LemmaStemTokenizer(),
                         strip_accents = 'unicode',
                         stop_words = stop_words,
                         lowercase = True,
#                          max_df = 10,
                         min_df = 2)

X = tf_cv1.fit_transform(df['prophecy'])

tf_cv1_df = pd.DataFrame(X.toarray(), columns = tf_cv1.get_feature_names())

/Users/charlieyaris/.venvs/lpthw/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'abov', 'accord', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sen', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


In [60]:
tf_cv1_df.head()

,abandon,abbey,abbot,able,above,abundance,abyss,accompanied,accomplish,accord,accuse,achieved,act,action,adherent,adria,adriatic,adulterer,advance,adversary,advice,advise,aegean,afar,affair,afflicted,afire,afraid,africa,african,afterwards,age,agen,agreement,aid,air,alas,alba,ale,alive,allied,alone,alps,altered,ambassador,ambush,amidst,ancient,ancona,anew,anger,anglican,angry,any,animal,annemark,anointed,another,answer,antibes,antique,antwerp,apennine,appear,appease,appointed,approach,april,aquarius,aqueduct,aquilon,aquitaine,arab,arabia,arch,area,aries,arise,arles,arm,armenia,armies,armor,arras,array,arrive,arrow,art,artemis,ashes,asia,ask,asleep,aspect,ass,assail,assault,assemble,assemble,assistance,asti,astonish,attack,attain,attended,attentive,auch,augsburg,ausonia,author,autun,avenge,avignon,await,away,azure,babylon,bad,baggage,balance,bale,band,banish,bank,banner,banquet,bar,barbaric,barbarian,barcelona,bark,barley,barrel,bastard,battle,battlefield,bayonne,bazas,beak,bear,beard,bearer,beast,beautiful,because,became,bed,bee,before,begin,belgian,believe,bell,belly,beneath,besieged,betray,beware,bezier,bigorre,bird,bit,bitter,black,blade,blame,blaye,bleed,bleedy,blind,blois,blond,blood,blow,blue,boar,boats,bodies,bold,bonds,bone,booty,bordeaux,bordelais,bordering,born,bosom,bound,bourg,bowing,brabant,brain,branch,brass,brave,breach,bread,break,bresse,breton,brick,bride,bridge,bright,brindisi,bring,britain,british,brittany,broken,brother,brought,bruges,brussels,built,burgundy,buried,burn,butcher,byzantine,byzantium,cadiz,cage,cahors,calais,calamities,calculated,camel,campania,cancer,candle,cannon,canton,capet,capricorn,captive,capture,carcassone,carcassonne,carmania,carpentras,carried,casks,cast,castle,castor,castres,cattle,cause,cave,cease,celebrate,celestial,celts,celtic,censored,centuries,chain,chalice,chalon,change,charge,chartres,chase,chief,child,choked,chosen,christian,church,chyren,cities,citizens,civil,claimed,clay,clear,clergy,climate,clip,cloaked,close,cloth,cloud,club,coast,cock,cold,collar,cologne,colonna,color,column,combat,come,commit,company,complain,complete,conceal,concubine,condemn,condom,conduct,confined,conflict,confuse,conjoined,conjunction,conquer,conqueror,consecrate,consent,console,conspirator,contented,contrary,convert,cope,cordoba,corinth,corner,corsica,council,counsel,countermanded,countries,course,court,cover,coward,crafty,created,crescent,cries,crook,cross,crowd,crown,cruel,crusader,crushed,cunning,cup,custom,cut,cycle,cymbals,cyprus,dagger,dalmatian,damage,danger,danube,dare,dark,dart,daughter,day,dead,death,debated,decay,deceit,deceitd,decline,deed,deep,defeated,defend,degree,delay,deliverance,deliverance,demand,denied,depart,deprive,descend,desert,deserved,desolated,despair,destroy,devastate,device,devour,die,different,digging,diminish,direct,discord,discover,dishonor,disloyal,disorder,dispute,distant,distress,disturbances,diverse,divide,divine,doe,dogs,dole,double,double,doubt,drag,draw,dread,dream,dressed,dry,drink,drive,driven,drizzle,drought,drowned,drunk,dryness,duel,duke,durance,during,dusky,dust,eagle,earth,earthquake,east,easter,eastern,eat,ebro,eclipse,edge,edict,efforts,egypt,elder,elect,elevate,elite,eloquence,emaciated,eminence,emperor,empire,emptied,enclose,encompassed,end,enemies,england,english,engrave,ennoble,enormous,enslaved,enter,enterprise,entire,envy,ephesus,equal,erect,escape,estate,eternal,etruria,europe,every,everlasting,everyone,everything,everywhere,evil,evreux,exalted,excellence,excess,execute,exerted,exhausted,exile,existence,expect,expelled,expose,extend,exterminate,extinguished,extreme,eye,face,fact,faenza,fail,faint,fair,faith,fall,fallen,false,fame,family,famine,fantasy,far,fast,fate,father,fault,favor,fear,feather,february,feet,feigned,felled,female,ferrara,festival,field,fierce,fifth,fight,finish,fish,fixed,flame,flanders,flash,fled,fled,fleece,fleet,flemings,flesh,fleur,flies,flock,flood,florence,flourish,flow,foix,fold,follies,follow,food,foot,force,forebear

In [23]:
len(tf_cv1_df.columns)

1533

In [24]:
for column in tf_cv1_df.columns:
    for i, stem_index in enumerate(tokenized_df_group['stem']):
        if column == stem_index:
            tf_cv1_df.rename({column: tokenized_df_group['original_word'][i]}, axis = 1, inplace = True)

In [25]:
len(tf_cv1_df.columns)

1533

In [26]:
# list(tf_cv1_df.columns)[2400:]

In [27]:
word_dic = {}
for i, word in enumerate(tf_cv1_df.columns):
    word_dic[word] = i

In [28]:
len(word_dic)

1525

In [29]:
counts = sparse.csr_matrix(tf_cv1_df.values.T)

In [30]:
counts.shape

(1533, 942)

In [31]:
corpus = matutils.Sparse2Corpus(counts)
print(corpus)

In [32]:
id2word = dict((v, k) for k, v in word_dic.items())

In [33]:
len(id2word)

1525

In [34]:
lda = models.LdaModel(corpus = corpus, num_topics = 2, id2word = id2word, passes = 100)

In [35]:
lda.print_topics()

[(0,
  '0.007*"come" + 0.005*"great" + 0.005*"king" + 0.004*"temple" + 0.004*"fail" + 0.004*"roman" + 0.004*"seven" + 0.003*"bridge" + 0.003*"vex" + 0.003*"fled"'),
 (1,
  '0.013*"great" + 0.007*"die" + 0.006*"land" + 0.006*"near" + 0.005*"very" + 0.005*"place" + 0.005*"sea" + 0.005*"king" + 0.005*"long" + 0.005*"bleed"')]

In [36]:
lda_corpus = lda[corpus]
print(lda_corpus)

In [37]:
lda_docs = [doc for doc in lda_corpus]

In [38]:
lda_docs[0:5]

[[(0, 0.1313619), (1, 0.8686381)],
 [(0, 0.12615052), (1, 0.8738495)],
 [(0, 0.18754648), (1, 0.8124535)],
 [(0, 0.13185), (1, 0.86815)],
 [(0, 0.609798), (1, 0.390202)]]

In [39]:
stop_word_list = ['able', 'according', 'aix', 'la', 'dy', 'having', 'l', 'le',
                  'li', 'o', 's', "’", 'very', 'like', 'sens', 'did', "'",
                  "'a", "'i", "'k", "'l", "'o", "'s", ',', '[', ']']
stop_words = text.ENGLISH_STOP_WORDS.union(stop_word_list)
tf_cv2 = TfidfVectorizer(tokenizer = LemmaStemTokenizer(),
                         ngram_range = (2, 2),
                         strip_accents = 'unicode',
                         stop_words = stop_words,
                         lowercase = True,
#                          max_df = 10,
                         min_df = 2)

X = tf_cv2.fit_transform(df['prophecy'])

tf_cv2_df = pd.DataFrame(X.toarray(), columns = tf_cv2.get_feature_names())

/Users/charlieyaris/.venvs/lpthw/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abl', 'abov', 'accord', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sen', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  sorted(inconsistent))


In [40]:
tf_cv2_df.head()

,abandon place,act deceit,adriat sea,aegean sea,agreement broken,ala great,alp cock,apennin mountain,appear earth,armi enter,armi led,armi rout,armi sea,arriv late,arrow shot,author great,away princ,babylon great,bad time,barbarian bleed,barbarian empir,barbarian fleet,battl given,beard star,becaus die,becaus great,becom tyranni,becom veri,befor assault,befor citi,befor die,befor great,befor long,befor mar,betray citi,betray great,bird prey,black white,bleed come,bleed die,bleed forc,bleed land,bleed long,bleed milk,bleed shed,bleed wound,bodi die,bodi soul,booti divid,born near,born obscur,born teeth,breton norman,bridg broken,british isl,broken citi,burn great,byzantin chief,captur die,captur hostag,carri prize,carri veri,castor pollux,caus bleed,caus evil,caus great,caus steel,celtic armi,celtic nation,chang great,chang heart,chang law,chang veri,chase bird,chief armi,chief murder,child born,chyren selin,citi besieg,citi deliv,citi great,clip wing,cock eagl,come die,come fail,come far,come great,come grow,come high,come king,come late,come night,come troubl,come veri,come vex,commit great,complet great,conflict given,conflict great,conflict near,conflict veri,conquer itali,corinth ephesus,cover bleed,cover earth,cri great,cri howl,cri tear,cross alp,cross pursu,crown great,cruel die,custom law,cut piec,deed great,deed place,deep sea,deliv great,demand relief,die becaus,die captur,die cruel,die drown,die franc,die great,die hand,die late,die soon,die veri,divers place,divin word,draw near,dream king,drive away,earth quak,earth trembl,easter templ,elev veri,empir transfer,enemi enemi,enemi outsid,enter citi,enter great,enterpris loss,entir sex,extinguish great,face anoint,fail time,faith law,faith promis,fall great,fall high,fame prais,fame renown,famin die,famin drought,famin plagu,famin steel,far away,far great,far realm,far rhone,father son,fear barbarian,feet hand,ferrara turin,field town,fifth great,flander ghent,fleet near,fleur lis,flow bleed,follow die,forc pass,foreign armi,foreign nation,foreign place,franc make,friend great,gallic bleed,gallic fleet,gallic king,gallic peopl,genoa nice,ghent brussel,gnaw bone,gold azur,gold silver,good fortun,good natur,good old,good omen,govern great,great armi,great asia,great assembl,great band,great brother,great captur,great chief,great chyren,great citi,great countri,great damag,great discord,great empir,great evil,great famin,great flee,great fleet,great flood,great forc,great good,great hail,great hung,great insult,great king,great ladi,great led,great light,great lord,great lose,great man,great master,great monarch,great murder,great nation,great neptun,great new,great number,great ocean,great peopl,great pestil,great pillag,great place,great plagu,great pontiff,great prelat,great princ,great rage,great river,great roman,great scandal,great selin,great slaughter,great soon,great suit,great terror,great thing,great troop,great wall,great war,guard betray,guard deceitd,half pig,hand come,hand great,hand vex,heaven earth,help come,hideous monster,high low,high mountain,high place,high realm,hold great,holi law,holi templ,honor rich,hope hand,horribl throne,hostag come,human bleed,hundr seven,hundr thousand,hunger thirst,infinit number,iron cage,issu great,join hold,join togeth,king byzantium,king come,king enter,king faith,king isl,king join,king kill,king new,king princ,king receiv,king support,king want,lake geneva,land great,land land,land sea,land stain,land trembl,late great,lead armi,lead flee,lead great,led captur,life die,light hors,ligurian sea,lion eagl,littl great,lock insid,london feign,long centuri,long die,long rain,long rest,long time,low high,mahometan peopl,make chang,make defend,make stop,make war,make way,man die,mani centuri,mar jupit,mar mercuri,march ancona,marin citi,marin fleet,marvel deed,middl crowd,militari hand,monarch chase,monster born,monster seen,moon complet,mountain great,mountain trembl,napl palermo,naval battl,near bank,near bridg,near gate,

In [41]:
len(tf_cv2_df.columns)

536

In [42]:
column_df = deepcopy(pd.DataFrame(tf_cv2_df.columns))
column_df[1] = [item[1] for item in column_df[0].str.split(' ')]
column_df[0] = [item[0] for item in column_df[0].str.split(' ')]

In [43]:
column_df.head()

,0,1
0,abandon,place
1,act,deceit
2,adriat,sea
3,aegean,sea
4,agreement,broken


In [44]:
for i, column in enumerate(column_df[0]):
    for z, stem_index in enumerate(tokenized_df_group['stem']):
        if column == stem_index:
            column_df[0].loc[i] = tokenized_df_group['original_word'][z]
            
for i, column in enumerate(column_df[1]):
    for z, stem_index in enumerate(tokenized_df_group['stem']):
        if column == stem_index:
            column_df[1].loc[i] = tokenized_df_group['original_word'][z]

In [45]:
column_df.head()

,0,1
0,abandon,place
1,act,deceit
2,adriatic,sea
3,aegean,sea
4,agreement,broken


In [46]:
for i, column in enumerate(tf_cv2_df.columns):
    tf_cv2_df.rename({column: column_df[0][i] + " " + column_df[1][i]}, axis = 1, inplace = True)

In [47]:
tf_cv2_df.head()

,abandon place,act deceit,adriatic sea,aegean sea,agreement broken,alas great,alps cock,apennine mountain,appear earth,armies enter,armies led,armies rout,armies sea,arrive late,arrow shot,author great,away prince,babylon great,bad time,barbarian bleed,barbarian empire,barbarian fleet,battle given,beard star,because die,because great,became tyranny,became very,before assault,before cities,before die,before great,before long,before mars,betray cities,betray great,bird prey,black white,bleed come,bleed die,bleed force,bleed land,bleed long,bleed milk,bleed shed,bleed wounded,bodies die,bodies soul,booty divide,born near,born obscure,born teeth,breton norman,bridge broken,british isle,broken cities,burn great,byzantine chief,capture die,capture hostage,carried prize,carried very,castor pollux,cause bleed,cause evil,cause great,cause steel,celtic armies,celtic nation,change great,change heart,change law,change very,chase bird,chief armies,chief murder,child born,chyren selin,cities besieged,cities deliver,cities great,clip wing,cock eagle,come die,come fail,come far,come great,come grow,come high,come king,come late,come night,come trouble,come very,come vex,commit great,complete great,conflict given,conflict great,conflict near,conflict very,conquer italy,corinth ephesus,cover bleed,cover earth,cries great,cries howl,cries tears,cross alps,cross pursue,crown great,cruel die,custom law,cut piece,deed great,deed place,deep sea,deliver great,demand relief,die because,die capture,die cruel,die drowned,die france,die great,die hand,die late,die soon,die very,diverse place,divine word,draw near,dream king,drive away,earth quake,earth tremble,easter temple,elevate very,empire transferred,enemies enemies,enemies outside,enter cities,enter great,enterprise loss,entire sex,extinguished great,face anointed,fail time,faith law,faith promise,fall great,fall high,fame praise,fame renown,famine die,famine drought,famine plague,famine steel,far away,far great,far realm,far rhone,father son,fear barbarian,feet hand,ferrara turin,field town,fifth great,flanders ghent,fleet near,fleur lis,flow bleed,follow die,force pass,foreign armies,foreign nation,foreign place,france made,friend great,gallic bleed,gallic fleet,gallic king,gallic people,genoa nice,ghent brussels,gnaw bone,gold azure,gold silver,good fortune,good natured,good old,good omen,govern great,great armies,great asia,great assemble,great band,great brother,great capture,great chief,great chyren,great cities,great countries,great damage,great discord,great empire,great evil,great famine,great fled,great fleet,great flood,great force,great good,great hail,great hung,great insult,great king,great ladies,great led,great light,great lord,great lose,great man,great master,great monarch,great murder,great nation,great neptune,great new,great number,great ocean,great people,great pestilence,great pillage,great place,great plague,great pontiff,great prelate,great prince,great rage,great river,great roman,great scandal,great selin,great slaughter,great soon,great suite,great terror,great thing,great troop,great wall,great war,guard betray,guard deceitd,half pig,hand come,hand great,hand vex,heaven earth,help come,hideous monster,high low,high mountain,high place,high realm,held great,holy law,holy temple,honor riches,hope hand,horrible throne,hostage come,human bleed,hundred seven,hundred thousand,hunger thirst,infinite number,iron cage,issued great,join held,join together,king byzantium,king come,king enter,king faith,king isle,king join,king kill,king new,king prince,king receive,king support,king want,lake geneva,land great,land land,land sea,land stained,land tremble,late great,lead armies,lead fled,lead great,led capture,life die,light horse,ligurian sea,lion eagle,little great,lock inside,london feigned,long centuries,long die,long rain,long rest,long time,low high,mahometan people,made change,made defend,made stop,made war,made way,man die,many centuries,mars jupiter,mars mercury,march anco

In [48]:
word_dic = {}
for i, word in enumerate(tf_cv2_df.columns):
    word_dic[word] = i

In [49]:
len(word_dic)

536

In [50]:
counts = sparse.csr_matrix(tf_cv2_df.values.T)

In [51]:
counts.shape

(536, 942)

In [52]:
corpus = matutils.Sparse2Corpus(counts)
print(corpus)

In [53]:
id2word = dict((v, k) for k, v in word_dic.items())

In [54]:
len(id2word)

536

In [55]:
lda = models.LdaModel(corpus = corpus, num_topics = 2, id2word = id2word, passes = 100)

In [56]:
lda.print_topics()

[(0,
  '0.013*"very great" + 0.009*"great armies" + 0.009*"great cities" + 0.007*"pyrenees mountain" + 0.006*"seven month" + 0.006*"great number" + 0.006*"great empire" + 0.006*"stained bleed" + 0.005*"conflict near" + 0.005*"near river"'),
 (1,
  '0.018*"long time" + 0.016*"land sea" + 0.007*"shaven head" + 0.006*"seven day" + 0.006*"seven year" + 0.005*"very near" + 0.005*"cut piece" + 0.005*"very far" + 0.005*"far away" + 0.005*"conflict very"')]

In [57]:
lda_corpus = lda[corpus]
print(lda_corpus)

In [58]:
lda_docs = [doc for doc in lda_corpus]

In [59]:
lda_docs[0:5]

[[(0, 0.5), (1, 0.5)],
 [(0, 0.5), (1, 0.5)],
 [(0, 0.23049906), (1, 0.769501)],
 [(0, 0.25541002), (1, 0.74459)],
 [(0, 0.7394724), (1, 0.26052764)]]

In [ ]:
lsa = TruncatedSVD(5, algorithm = 'arpack')
print(lsa)

In [ ]:
X_lsa = lsa.fit_transform(sparse.csr_matrix(tf_cv2_df.values.T))
X_lsa = Normalizer(copy = False).fit_transform(X_lsa)

In [ ]:
lsa.explained_variance_ratio_

In [ ]:
def display_topics(model, feature_names, no_top_words, topic_names = None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '", topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1: -1]]))

In [ ]:
display_topics(lsa, list(tf_cv2_df.columns), 10)